In [1]:
import bigframes

# On the instance where you are running jupyter,
# authenticate with gcloud first:
#
#     gcloud auth application-default login

# TODO(swast): How should folks connect when using colab
# when the runtime isn't running locally with access to
# credentials from the command line?
engine = bigframes.connect()

In [2]:
df = engine.read_gbq("bigquery-public-data.chicago_taxi_trips.taxi_trips")[[
    "trip_end_timestamp",
    "pickup_location",
    "trip_miles",
    "trip_total",
]]

In [3]:
df["trip_miles"].round(decimals=2)._value

r0 := BigQueryTable: bigquery-public-data.chicago_taxi_trips.taxi_trips
  unique_key             string
  taxi_id                string
  trip_start_timestamp   timestamp
  trip_end_timestamp     timestamp
  trip_seconds           int64
  trip_miles             float64
  pickup_census_tract    int64
  dropoff_census_tract   int64
  pickup_community_area  int64
  dropoff_community_area int64
  fare                   float64
  tips                   float64
  tolls                  float64
  extras                 float64
  trip_total             float64
  payment_type           string
  company                string
  pickup_latitude        float64
  pickup_longitude       float64
  pickup_location        string
  dropoff_latitude       float64
  dropoff_longitude      float64
  dropoff_location       string

Selection[r0]
  selections:
    trip_miles: Round(r0.trip_miles, digits=2)

In [3]:
df.head(20)

,trip_end_timestamp,pickup_location,trip_miles,trip_total
0,2013-01-03 06:15:00,None,0.7,4.65
1,2013-01-03 06:45:00,None,0.2,4.05
2,2013-01-03 06:45:00,None,0.5,4.65
3,2013-01-02 12:45:00,None,0.9,5.45
4,2013-01-02 13:00:00,None,1.1,6.05
5,2013-01-02 12:15:00,None,0.8,5.65
6,2013-01-02 12:15:00,None,1.2,6.25
7,2013-01-02 11:45:00,None,1.2,5.85
8,2013-01-02 13:15:00,None,1.4,7.25
9,2013-03-17 10:15:00,None,2.2,10.44


In [4]:
df['trip_total']

In [5]:
df.head()['trip_total']

0    4.65
1    4.05
2    4.65
3    5.45
4    6.05
Name: trip_total, dtype: float64

In [6]:
df['trip_total'].head()

AttributeError: 'Series' object has no attribute 'head'